In [3]:
"""
References:
https://arxiv.org/pdf/1512.03385.pdf
https://arxiv.org/pdf/1604.04112v4.pdf
"""
import numpy as np
from tensorflow.keras.layers import Add, Dense, Conv2D, BatchNormalization,ReLU
from tensorflow.keras.layers import Activation, AveragePooling2D, Input, Flatten,MaxPool2D
from tensorflow.keras.callbacks import LearningRateScheduler,TensorBoard,EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model,load_model
from cutout_generator import CutoutImageDataGenerator
from mixup_generator import MixupGenerator
from cutmix_generator import CutMixGenerator

# hyperparameters

In [4]:
batch_size = 128
epochs = 120
n_classes = 100
learning_rate = 0.1

# load data

In [5]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)           #(50000, 32, 32, 3) (50000, 100) (10000, 32, 32, 3) (10000,100)

(50000, 32, 32, 3) (50000, 100) (10000, 32, 32, 3) (10000, 100)


# normalization

In [6]:
def get_mean_std(images):
    images = images.astype(np.float32)
    mean_vals = ()
    std_vals = ()
    for i in range(images.shape[-1]):
        mean_vals += (np.mean(images[:, :, :, i]),)
        std_vals += (np.std(images[:, :, :, i]),)
    return mean_vals, std_vals

mean_vals, std_vals = get_mean_std(x_train)
print(mean_vals)
print(std_vals)

(129.30428, 124.07023, 112.43411)
(68.17024, 65.391785, 70.4184)


In [7]:
def normalize(images, mean_vals, std_vals):
    images = images.astype(np.float32)
    for i in range(images.shape[-1]):
        images[:, :, :, i] = (images[:, :, :, i] - mean_vals[i])/std_vals[i]
    return images

x_train = normalize(x_train, mean_vals, std_vals)
x_test = normalize(x_test, mean_vals, std_vals)
print(x_test.max(), x_test.min())

2.0245545 -1.8973366


# divide train and val

In [8]:
val_frac = 0.2
perm_ids = np.random.permutation(x_train.shape[0])
val_ids = perm_ids[:int(val_frac*x_train.shape[0])]
train_ids = perm_ids[int(val_frac*x_train.shape[0]):]
x_val, y_val = x_train[val_ids], y_train[val_ids]
x_train, y_train = x_train[train_ids], y_train[train_ids]
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(40000, 32, 32, 3) (40000, 100) (10000, 32, 32, 3) (10000, 100)


# network

In [9]:
class Resnet:
    def __init__(self, size=56, stacks=3, starting_filter=16):
        self.size = size
        self.stacks = stacks
        self.starting_filter = starting_filter
        self.residual_blocks = (size - 2) // 6
        
    def get_model(self, input_shape=(32, 32, 3), n_classes=100):
        n_filters = self.starting_filter

        inputs = Input(shape=input_shape)
        network = self.layer(inputs, n_filters)
        network = self.stack(network, n_filters, True)

        for _ in range(self.stacks - 1):
            n_filters *= 2
            network = self.stack(network, n_filters)

        network = Activation('elu')(network)
        pd=network.shape[1]
        network = AveragePooling2D(pool_size=(pd,pd))(network)
        network = Flatten()(network)
        outputs = Dense(n_classes, activation='softmax', 
                        kernel_initializer='he_normal')(network)

        model = Model(inputs=inputs, outputs=outputs)

        return model
    
    def stack(self, inputs, n_filters, first_stack=False):
        stack = inputs

        if first_stack:
            stack = self.identity_block(stack, n_filters)
        else:
            stack = self.convolution_block(stack, n_filters)

        for _ in range(self.residual_blocks - 1):
            stack = self.identity_block(stack, n_filters)

        return stack
    
    def identity_block(self, inputs, n_filters):
        shortcut = inputs

        block = self.layer(inputs, n_filters, normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        block = Add()([shortcut, block])

        return block

    def convolution_block(self, inputs, n_filters, strides=2):
        shortcut = inputs

        block = self.layer(inputs, n_filters, strides=strides,
                           normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        shortcut = self.layer(shortcut, n_filters,
                              kernel_size=1, strides=strides,
                              activation=None)

        block = Add()([shortcut, block])

        return block
    
    def layer(self, inputs, n_filters, kernel_size=3,
              strides=1, activation='elu', normalize_batch=True):
    
        convolution = Conv2D(n_filters, kernel_size=kernel_size,
                             strides=strides, padding='same',
                             kernel_initializer="he_normal",
                             kernel_regularizer=l2(1e-4))

        x = convolution(inputs)

        if normalize_batch:
            x = BatchNormalization()(x)

        if activation is not None:
            x = Activation(activation)(x)

        return x
    

In [10]:
resnet = Resnet()

In [11]:
model = resnet.get_model()

optimizer = SGD(lr=learning_rate, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization_v1[0][0]     
_____________________________________

# Train Network

### baseline witout data augmentation

In [ ]:
log= TensorBoard(log_dir="./logs/baseline")

callbacks = [log,EarlyStopping(monitor = 'val_loss', patience = 13), 
             ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 6, verbose = 1，min_lr=1e-4)]
#datagen = ImageDataGenerator(width_shift_range=0.15,
#                             height_shift_range=0.15,
#                            horizontal_flip=True)
datagen = ImageDataGenerator()
datagen.fit(x_train)

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          validation_data=(x_val, y_val),
          steps_per_epoch=x_train.shape[0] // batch_size,
          epochs=epochs,
          callbacks=callbacks)

model.save('model_withoutaug.h5')

### baseline

In [15]:
model = resnet.get_model()

optimizer = SGD(lr=learning_rate, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

log= TensorBoard(log_dir="./logs/baseline_1")

callbacks = [log,EarlyStopping(monitor = 'val_loss', patience = 13), 
             ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 6, verbose = 1,min_lr=1e-5)]

datagen = ImageDataGenerator(width_shift_range=0.15,
                             height_shift_range=0.15,
                             horizontal_flip=True)
datagen.fit(x_train)

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          validation_data=(x_val, y_val),
          steps_per_epoch=x_train.shape[0] // batch_size,
          epochs=epochs,
          callbacks=callbacks)

model.save('model_baseline_1.h5')

Epoch 1/120
313/313 [==============================] - 58s 184ms/step - loss: 4.2958 - acc: 0.1075 - val_loss: 6.2511 - val_acc: 0.0673
Epoch 2/120
313/313 [==============================] - 52s 168ms/step - loss: 3.6654 - acc: 0.2040 - val_loss: 3.5579 - val_acc: 0.2278
Epoch 3/120
313/313 [==============================] - 54s 171ms/step - loss: 3.2917 - acc: 0.2699 - val_loss: 3.1450 - val_acc: 0.2936
Epoch 4/120
313/313 [==============================] - 54s 173ms/step - loss: 2.9961 - acc: 0.3282 - val_loss: 2.9291 - val_acc: 0.3420
Epoch 5/120
313/313 [==============================] - 54s 173ms/step - loss: 2.7848 - acc: 0.3719 - val_loss: 2.7893 - val_acc: 0.3764
Epoch 6/120
313/313 [==============================] - 54s 174ms/step - loss: 2.6044 - acc: 0.4107 - val_loss: 2.9430 - val_acc: 0.3527
Epoch 7/120
313/313 [==============================] - 55s 174ms/step - loss: 2.4692 - acc: 0.4394 - val_loss: 2.7923 - val_acc: 0.3868
Epoch 8/120
313/313 [===========================

Epoch 37/120
313/313 [==============================] - 54s 174ms/step - loss: 1.2632 - acc: 0.7951 - val_loss: 1.9342 - val_acc: 0.6418
Epoch 38/120
313/313 [==============================] - 56s 180ms/step - loss: 1.1272 - acc: 0.8305 - val_loss: 1.9125 - val_acc: 0.6476
Epoch 39/120
313/313 [==============================] - 55s 177ms/step - loss: 1.0703 - acc: 0.8433 - val_loss: 1.9371 - val_acc: 0.6514
Epoch 40/120
313/313 [==============================] - 56s 179ms/step - loss: 1.0212 - acc: 0.8548 - val_loss: 1.9085 - val_acc: 0.6528
Epoch 41/120
313/313 [==============================] - 56s 180ms/step - loss: 0.9851 - acc: 0.8613 - val_loss: 1.8909 - val_acc: 0.6604
Epoch 42/120
313/313 [==============================] - 56s 179ms/step - loss: 0.9540 - acc: 0.8668 - val_loss: 1.9455 - val_acc: 0.6478
Epoch 43/120
313/313 [==============================] - 56s 178ms/step - loss: 0.9232 - acc: 0.8750 - val_loss: 1.9784 - val_acc: 0.6483
Epoch 44/120
313/313 [===================

### Cutout

In [19]:
model = resnet.get_model()
optimizer = SGD(lr=learning_rate, momentum=0.9)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

log= TensorBoard(log_dir="./logs/cutout")

callbacks = [log,EarlyStopping(monitor = 'val_loss', patience = 13), 
             ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 6, verbose = 1,min_lr=1e-4)]

datagen = CutoutImageDataGenerator(width_shift_range=0.15,
                                   height_shift_range=0.15,
                                   horizontal_flip=True,
                                   cutout_mask_size=10)

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          validation_data=(x_val, y_val),
          steps_per_epoch=x_train.shape[0] // batch_size,
          epochs=epochs,
          callbacks=callbacks)

model.save('model_cutout.h5')

Epoch 1/120
312/312 [==============================] - 61s 194ms/step - loss: 4.3307 - acc: 0.1061 - val_loss: 4.7192 - val_acc: 0.1047
Epoch 2/120
312/312 [==============================] - 52s 167ms/step - loss: 3.7099 - acc: 0.1968 - val_loss: 3.5594 - val_acc: 0.2192
Epoch 3/120
312/312 [==============================] - 52s 167ms/step - loss: 3.3554 - acc: 0.2609 - val_loss: 3.1621 - val_acc: 0.3043
Epoch 4/120
312/312 [==============================] - 53s 169ms/step - loss: 3.1178 - acc: 0.3034 - val_loss: 3.0320 - val_acc: 0.3283
Epoch 5/120
312/312 [==============================] - 53s 171ms/step - loss: 2.9250 - acc: 0.3421 - val_loss: 2.9952 - val_acc: 0.3405
Epoch 6/120
312/312 [==============================] - 54s 172ms/step - loss: 2.7840 - acc: 0.3704 - val_loss: 2.9505 - val_acc: 0.3469
Epoch 7/120
312/312 [==============================] - 54s 174ms/step - loss: 2.6472 - acc: 0.3990 - val_loss: 2.7740 - val_acc: 0.3812
Epoch 8/120
312/312 [===========================

Epoch 37/120
312/312 [==============================] - 57s 181ms/step - loss: 1.3148 - acc: 0.7554 - val_loss: 1.7783 - val_acc: 0.6545
Epoch 38/120
312/312 [==============================] - 58s 184ms/step - loss: 1.2782 - acc: 0.7645 - val_loss: 1.7881 - val_acc: 0.6501
Epoch 39/120
312/312 [==============================] - 59s 188ms/step - loss: 1.2559 - acc: 0.7686 - val_loss: 1.7600 - val_acc: 0.6550
Epoch 40/120
312/312 [==============================] - 60s 193ms/step - loss: 1.2218 - acc: 0.7738 - val_loss: 1.7770 - val_acc: 0.6530
Epoch 41/120
312/312 [==============================] - 58s 186ms/step - loss: 1.2076 - acc: 0.7764 - val_loss: 1.8025 - val_acc: 0.6458
Epoch 42/120
312/312 [==============================] - 58s 187ms/step - loss: 1.1926 - acc: 0.7781 - val_loss: 1.7996 - val_acc: 0.6471
Epoch 43/120
312/312 [==============================] - 58s 186ms/step - loss: 1.1720 - acc: 0.7842 - val_loss: 1.7712 - val_acc: 0.6547
Epoch 44/120
312/312 [===================

### Mixup

In [13]:
model = resnet.get_model()
optimizer = SGD(lr=learning_rate, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

log= TensorBoard(log_dir="./logs/mixup")

callbacks = [log,EarlyStopping(monitor = 'val_loss', patience = 13), 
             ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, patience = 6, verbose = 1,min_lr=1e-5)]

datagen = ImageDataGenerator(width_shift_range=0.15,
                             height_shift_range=0.15,
                             horizontal_flip=True)

training_generator = MixupGenerator(x_train, y_train, batch_size=batch_size, alpha=0.4, datagen=datagen)()
model.fit(training_generator,
          steps_per_epoch=x_train.shape[0] // batch_size,
          validation_data=(x_val, y_val),
          epochs=epochs,
          callbacks=callbacks)

model.save('model_mixup.h5')

Epoch 1/120
312/312 [==============================] - 51s 163ms/step - loss: 4.5433 - acc: 0.0933 - val_loss: 4.7459 - val_acc: 0.0910
Epoch 2/120
312/312 [==============================] - 49s 155ms/step - loss: 4.0656 - acc: 0.1755 - val_loss: 3.6674 - val_acc: 0.2093
Epoch 3/120
312/312 [==============================] - 50s 160ms/step - loss: 3.7988 - acc: 0.2280 - val_loss: 3.4070 - val_acc: 0.2510
Epoch 4/120
312/312 [==============================] - 50s 161ms/step - loss: 3.5955 - acc: 0.2789 - val_loss: 3.1434 - val_acc: 0.2998
Epoch 5/120
312/312 [==============================] - 50s 161ms/step - loss: 3.4305 - acc: 0.3100 - val_loss: 2.8690 - val_acc: 0.3520
Epoch 6/120
312/312 [==============================] - 51s 165ms/step - loss: 3.2972 - acc: 0.3445 - val_loss: 2.7320 - val_acc: 0.3791
Epoch 7/120
312/312 [==============================] - 51s 165ms/step - loss: 3.1946 - acc: 0.3711 - val_loss: 2.6875 - val_acc: 0.3905
Epoch 8/120
312/312 [===========================

Epoch 37/120
312/312 [==============================] - 51s 165ms/step - loss: 2.4710 - acc: 0.6023 - val_loss: 1.9589 - val_acc: 0.5708
Epoch 38/120
312/312 [==============================] - 51s 164ms/step - loss: 2.4559 - acc: 0.6057 - val_loss: 1.9265 - val_acc: 0.5810
Epoch 39/120
312/312 [==============================] - 51s 164ms/step - loss: 2.4564 - acc: 0.6089 - val_loss: 1.9374 - val_acc: 0.5772
Epoch 40/120
312/312 [==============================] - 52s 165ms/step - loss: 2.4342 - acc: 0.6131 - val_loss: 1.9513 - val_acc: 0.5753
Epoch 41/120
312/312 [==============================] - 52s 166ms/step - loss: 2.4444 - acc: 0.6131 - val_loss: 2.0188 - val_acc: 0.5634
Epoch 42/120
10000/10000 [==============================] - 3s 325us/sample - loss: 1.9948 - acc: 0.5680

Epoch 00042: ReduceLROnPlateau reducing learning rate to 0.030000000447034835.
312/312 [==============================] - 55s 176ms/step - loss: 2.4392 - acc: 0.6155 - val_loss: 1.9880 - val_acc: 0.5680
Epoch 

312/312 [==============================] - 51s 163ms/step - loss: 1.7003 - acc: 0.8234 - val_loss: 1.4326 - val_acc: 0.6841
Epoch 108/120
10000/10000 [==============================] - 3s 330us/sample - loss: 1.4404 - acc: 0.6836

Epoch 00108: ReduceLROnPlateau reducing learning rate to 0.00024299999931827186.
312/312 [==============================] - 51s 165ms/step - loss: 1.6990 - acc: 0.8260 - val_loss: 1.4322 - val_acc: 0.6836
Epoch 109/120
312/312 [==============================] - 51s 165ms/step - loss: 1.6932 - acc: 0.8277 - val_loss: 1.4312 - val_acc: 0.6835
Epoch 110/120
312/312 [==============================] - 51s 165ms/step - loss: 1.7084 - acc: 0.8251 - val_loss: 1.4318 - val_acc: 0.6837
Epoch 111/120
312/312 [==============================] - 51s 164ms/step - loss: 1.7002 - acc: 0.8254 - val_loss: 1.4311 - val_acc: 0.6840
Epoch 112/120
312/312 [==============================] - 51s 164ms/step - loss: 1.6918 - acc: 0.8245 - val_loss: 1.4286 - val_acc: 0.6842
Epoch 113/12

### Cutmix

In [18]:
model = resnet.get_model()
optimizer = SGD(lr=learning_rate, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

log= TensorBoard(log_dir="./logs/cutmix")

callbacks = [log,EarlyStopping(monitor = 'val_loss', patience = 13), 
             ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 6, verbose = 1,min_lr=1e-5)]

datagen = ImageDataGenerator(width_shift_range=0.15,
                             height_shift_range=0.15,
                             horizontal_flip=True)

training_generator = CutMixGenerator(x_train, y_train, batch_size=batch_size, alpha=0.8, datagen=datagen)()
model.fit(training_generator,
          steps_per_epoch=x_train.shape[0] // batch_size,
          validation_data=(x_val, y_val),
          epochs=epochs,
          callbacks=callbacks)

model.save('model_cutmix.h5')

Epoch 1/120
312/312 [==============================] - 55s 177ms/step - loss: 4.7882 - acc: 0.0540 - val_loss: 4.2896 - val_acc: 0.1091
Epoch 2/120
312/312 [==============================] - 52s 166ms/step - loss: 4.4708 - acc: 0.0964 - val_loss: 3.9187 - val_acc: 0.1483
Epoch 3/120
312/312 [==============================] - 52s 167ms/step - loss: 4.3109 - acc: 0.1269 - val_loss: 3.6176 - val_acc: 0.2066
Epoch 4/120
312/312 [==============================] - 52s 168ms/step - loss: 4.2179 - acc: 0.1400 - val_loss: 3.4858 - val_acc: 0.2358
Epoch 5/120
312/312 [==============================] - 54s 172ms/step - loss: 4.0725 - acc: 0.1674 - val_loss: 3.3327 - val_acc: 0.2521
Epoch 6/120
312/312 [==============================] - 52s 168ms/step - loss: 4.0165 - acc: 0.1780 - val_loss: 3.1457 - val_acc: 0.2913
Epoch 7/120
312/312 [==============================] - 52s 168ms/step - loss: 3.9188 - acc: 0.1954 - val_loss: 3.0253 - val_acc: 0.3084
Epoch 8/120
312/312 [===========================

Epoch 37/120
312/312 [==============================] - 51s 165ms/step - loss: 3.1342 - acc: 0.4022 - val_loss: 2.1634 - val_acc: 0.5075
Epoch 38/120
312/312 [==============================] - 51s 165ms/step - loss: 3.2109 - acc: 0.3923 - val_loss: 2.0822 - val_acc: 0.5247
Epoch 39/120
312/312 [==============================] - 51s 164ms/step - loss: 3.2556 - acc: 0.3758 - val_loss: 2.1055 - val_acc: 0.5259
Epoch 40/120
312/312 [==============================] - 51s 165ms/step - loss: 3.2339 - acc: 0.3757 - val_loss: 2.0897 - val_acc: 0.5197
Epoch 41/120
312/312 [==============================] - 51s 165ms/step - loss: 3.2144 - acc: 0.3690 - val_loss: 2.0381 - val_acc: 0.5411
Epoch 42/120
312/312 [==============================] - 51s 163ms/step - loss: 3.2640 - acc: 0.3687 - val_loss: 2.2521 - val_acc: 0.5147
Epoch 43/120
312/312 [==============================] - 51s 162ms/step - loss: 3.1818 - acc: 0.3959 - val_loss: 2.0942 - val_acc: 0.5380
Epoch 44/120
312/312 [===================

312/312 [==============================] - 52s 165ms/step - loss: 2.6869 - acc: 0.4941 - val_loss: 1.6621 - val_acc: 0.6317
Epoch 73/120
312/312 [==============================] - 51s 165ms/step - loss: 2.6730 - acc: 0.4903 - val_loss: 1.6277 - val_acc: 0.6350
Epoch 74/120
312/312 [==============================] - 51s 165ms/step - loss: 2.7091 - acc: 0.5021 - val_loss: 1.6658 - val_acc: 0.6333
Epoch 75/120
312/312 [==============================] - 51s 165ms/step - loss: 2.7181 - acc: 0.4794 - val_loss: 1.6724 - val_acc: 0.6356
Epoch 76/120
312/312 [==============================] - 52s 165ms/step - loss: 2.6685 - acc: 0.5063 - val_loss: 1.5835 - val_acc: 0.6450
Epoch 77/120
312/312 [==============================] - 52s 165ms/step - loss: 2.6533 - acc: 0.5072 - val_loss: 1.6284 - val_acc: 0.6345
Epoch 78/120
312/312 [==============================] - 51s 164ms/step - loss: 2.6729 - acc: 0.4848 - val_loss: 1.6390 - val_acc: 0.6262
Epoch 79/120
312/312 [==============================] 

Epoch 108/120
10000/10000 [==============================] - 3s 333us/sample - loss: 1.4645 - acc: 0.6782

Epoch 00108: ReduceLROnPlateau reducing learning rate to 0.0007999999448657036.
312/312 [==============================] - 51s 163ms/step - loss: 2.4612 - acc: 0.5418 - val_loss: 1.4657 - val_acc: 0.6782
Epoch 109/120
312/312 [==============================] - 51s 163ms/step - loss: 2.4715 - acc: 0.5398 - val_loss: 1.4240 - val_acc: 0.6854
Epoch 110/120
312/312 [==============================] - 51s 163ms/step - loss: 2.4510 - acc: 0.5377 - val_loss: 1.4200 - val_acc: 0.6846
Epoch 111/120
312/312 [==============================] - 51s 163ms/step - loss: 2.4117 - acc: 0.5660 - val_loss: 1.4302 - val_acc: 0.6850
Epoch 112/120
312/312 [==============================] - 51s 163ms/step - loss: 2.4854 - acc: 0.5416 - val_loss: 1.4285 - val_acc: 0.6859
Epoch 113/120
312/312 [==============================] - 51s 163ms/step - loss: 2.4430 - acc: 0.5557 - val_loss: 1.4079 - val_acc: 0.6873

# Evaluate

In [12]:
def evaluate(model,x_test,y_test):
    loss,acc=model.evaluate(x_test,y_test)
    return loss,acc

In [15]:
prnt = ('test loss {:0.4f}  test acc:{:0.4f}')

model_base=load_model("model_withoutaug.h5")
loss,acc=evaluate(model_base,x_test,y_test)
print("baseline without aug :"+prnt.format(loss,acc)+'\n')

model_baseline=load_model("model_baseline.h5")
loss,acc=evaluate(model_baseline,x_test,y_test)
print("baseline: "+prnt.format(loss,acc)+'\n')

model_cutout=load_model("model_cutout.h5")
loss,acc=evaluate(model_cutout,x_test,y_test)
print("baseline  with cutout: "+prnt.format(loss,acc)+'\n')

model_mixup=load_model("model_mixup.h5")
loss,acc=evaluate(model_mixup,x_test,y_test)
print("baseline  with mixup: "+prnt.format(loss,acc)+'\n')

model_cutmix=load_model("model_cutmix.h5")
loss,acc=evaluate(model_cutmix,x_test,y_test)
print("baseline with cutmix: "+prnt.format(loss,acc)+'\n')

10000/10000 [==============================] - 6s 638us/sample - loss: 2.6015 - acc: 0.5659
baseline without aug :test loss 2.6015  test acc:0.5659

10000/10000 [==============================] - 6s 647us/sample - loss: 1.9347 - acc: 0.6669
baseline: test loss 1.9347  test acc:0.6669

10000/10000 [==============================] - 7s 682us/sample - loss: 1.7301 - acc: 0.6745
baseline  with cutout: test loss 1.7301  test acc:0.6745

10000/10000 [==============================] - 7s 725us/sample - loss: 1.4141 - acc: 0.6896
baseline  with mixup: test loss 1.4141  test acc:0.6896

10000/10000 [==============================] - 8s 756us/sample - loss: 1.4149 - acc: 0.6923
baseline with cutmix: test loss 1.4149  test acc:0.6923



# Further more

In [17]:
def cutout(cutout_mask_size = 8):
    def cut(input_img):
        image = np.copy(input_img)
        mask_value = image.mean()
        
        
        h, w, _ = image.shape
        
        y = np.random.randint(h)
        x = np.random.randint(w)
        
        
        top = np.clip(y - cutout_mask_size // 2, 0, h)
        bottom  = np.clip(y + cutout_mask_size // 2, 0, h)
        left = np.clip(x - cutout_mask_size // 2, 0, w)
        right = np.clip(x + cutout_mask_size // 2, 0, w)
        
        image[top:bottom, left:right, :].fill(mask_value)
        return image

    return cut

## cutout + mixup

In [18]:
model = resnet.get_model()
optimizer = SGD(lr=learning_rate, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

log= TensorBoard(log_dir="./logs/cm")

callbacks = [log,EarlyStopping(monitor = 'val_loss', patience = 13), 
             ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 6, verbose = 1,min_lr=1e-5)]

datagen = ImageDataGenerator(width_shift_range=0.15,
                             height_shift_range=0.15,
                             horizontal_flip=True,
                             preprocessing_function=cutout(cutout_mask_size = 10))

training_generator = MixupGenerator(x_train, y_train, batch_size=batch_size, alpha=0.4, datagen=datagen)()
model.fit(training_generator,
          steps_per_epoch=x_train.shape[0] // batch_size,
          validation_data=(x_val, y_val),
          epochs=epochs,
          callbacks=callbacks)

model.save('model.cm.h5')

Epoch 1/120
312/312 [==============================] - 54s 174ms/step - loss: 4.5575 - acc: 0.0846 - val_loss: 4.7787 - val_acc: 0.0945
Epoch 2/120
312/312 [==============================] - 49s 156ms/step - loss: 4.1408 - acc: 0.1554 - val_loss: 3.9396 - val_acc: 0.1673
Epoch 3/120
312/312 [==============================] - 50s 159ms/step - loss: 3.9152 - acc: 0.2007 - val_loss: 3.5438 - val_acc: 0.2293
Epoch 4/120
312/312 [==============================] - 51s 162ms/step - loss: 3.7226 - acc: 0.2415 - val_loss: 3.2208 - val_acc: 0.2822
Epoch 5/120
312/312 [==============================] - 51s 163ms/step - loss: 3.5948 - acc: 0.2665 - val_loss: 2.9533 - val_acc: 0.3244
Epoch 6/120
312/312 [==============================] - 51s 165ms/step - loss: 3.4713 - acc: 0.2979 - val_loss: 2.8815 - val_acc: 0.3425
Epoch 7/120
312/312 [==============================] - 52s 165ms/step - loss: 3.3780 - acc: 0.3179 - val_loss: 2.7395 - val_acc: 0.3668
Epoch 8/120
312/312 [===========================

Epoch 37/120
312/312 [==============================] - 53s 169ms/step - loss: 2.6256 - acc: 0.5483 - val_loss: 1.9857 - val_acc: 0.5673
Epoch 38/120
312/312 [==============================] - 53s 169ms/step - loss: 2.6010 - acc: 0.5537 - val_loss: 1.9124 - val_acc: 0.5783
Epoch 39/120
312/312 [==============================] - 53s 169ms/step - loss: 2.6025 - acc: 0.5546 - val_loss: 1.9755 - val_acc: 0.5682
Epoch 40/120
312/312 [==============================] - 53s 169ms/step - loss: 2.6063 - acc: 0.5554 - val_loss: 2.1714 - val_acc: 0.5304
Epoch 41/120
312/312 [==============================] - 53s 169ms/step - loss: 2.5963 - acc: 0.5587 - val_loss: 1.9978 - val_acc: 0.5646
Epoch 42/120
312/312 [==============================] - 53s 169ms/step - loss: 2.5915 - acc: 0.5622 - val_loss: 1.9003 - val_acc: 0.5822
Epoch 43/120
312/312 [==============================] - 53s 169ms/step - loss: 2.5749 - acc: 0.5666 - val_loss: 2.0424 - val_acc: 0.5545
Epoch 44/120
312/312 [===================

312/312 [==============================] - 52s 165ms/step - loss: 2.0698 - acc: 0.7014 - val_loss: 1.5412 - val_acc: 0.6566
Epoch 73/120
312/312 [==============================] - 51s 165ms/step - loss: 2.0644 - acc: 0.7006 - val_loss: 1.4950 - val_acc: 0.6670
Epoch 74/120
312/312 [==============================] - 51s 164ms/step - loss: 2.0597 - acc: 0.7029 - val_loss: 1.5589 - val_acc: 0.6564
Epoch 75/120
312/312 [==============================] - 51s 164ms/step - loss: 2.0535 - acc: 0.7028 - val_loss: 1.5394 - val_acc: 0.6558
Epoch 76/120
312/312 [==============================] - 51s 164ms/step - loss: 2.0353 - acc: 0.7071 - val_loss: 1.5431 - val_acc: 0.6532
Epoch 77/120
312/312 [==============================] - 51s 164ms/step - loss: 2.0467 - acc: 0.7044 - val_loss: 1.5984 - val_acc: 0.6437
Epoch 78/120
312/312 [==============================] - 51s 164ms/step - loss: 2.0339 - acc: 0.7051 - val_loss: 1.5490 - val_acc: 0.6543
Epoch 79/120
10000/10000 [============================

## cutout + cutmix

In [27]:
model = resnet.get_model()
optimizer = SGD(lr=learning_rate, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

log= TensorBoard(log_dir="./logs/cc")

callbacks = [log,EarlyStopping(monitor = 'val_loss', patience = 13), 
             ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, patience = 6, verbose = 1,min_lr=1e-5)]

datagen = ImageDataGenerator(width_shift_range=0.15,
                             height_shift_range=0.15,
                             horizontal_flip=True,
                            preprocessing_function=cutout(cutout_mask_size = 10))

training_generator = CutMixGenerator(x_train, y_train, batch_size=batch_size, alpha=0.8, datagen=datagen)()
model.fit(training_generator,
          steps_per_epoch=x_train.shape[0] // batch_size,
          validation_data=(x_val, y_val),
          epochs=epochs,
          callbacks=callbacks)

model.save('model_cc.h5')

Epoch 1/120
312/312 [==============================] - 58s 187ms/step - loss: 4.7603 - acc: 0.0557 - val_loss: 5.8549 - val_acc: 0.0562
Epoch 2/120
312/312 [==============================] - 49s 159ms/step - loss: 4.4502 - acc: 0.1016 - val_loss: 3.8511 - val_acc: 0.1643
Epoch 3/120
312/312 [==============================] - 50s 160ms/step - loss: 4.2718 - acc: 0.1379 - val_loss: 3.6842 - val_acc: 0.1988
Epoch 4/120
312/312 [==============================] - 50s 160ms/step - loss: 4.1575 - acc: 0.1563 - val_loss: 3.2278 - val_acc: 0.2732
Epoch 5/120
312/312 [==============================] - 51s 162ms/step - loss: 4.0328 - acc: 0.1790 - val_loss: 3.2118 - val_acc: 0.2768
Epoch 6/120
312/312 [==============================] - 51s 164ms/step - loss: 3.9340 - acc: 0.1937 - val_loss: 2.9051 - val_acc: 0.3405
Epoch 7/120
312/312 [==============================] - 51s 164ms/step - loss: 3.8872 - acc: 0.2096 - val_loss: 2.9500 - val_acc: 0.3249
Epoch 8/120
312/312 [===========================

Epoch 37/120
312/312 [==============================] - 53s 169ms/step - loss: 3.2814 - acc: 0.3636 - val_loss: 2.3776 - val_acc: 0.4741
Epoch 38/120
312/312 [==============================] - 53s 169ms/step - loss: 3.1877 - acc: 0.3925 - val_loss: 2.1590 - val_acc: 0.5131
Epoch 39/120
312/312 [==============================] - 53s 169ms/step - loss: 3.2629 - acc: 0.3693 - val_loss: 2.0444 - val_acc: 0.5463
Epoch 40/120
10000/10000 [==============================] - 4s 374us/sample - loss: 2.1278 - acc: 0.5374

Epoch 00040: ReduceLROnPlateau reducing learning rate to 0.030000000447034835.
312/312 [==============================] - 69s 222ms/step - loss: 3.2653 - acc: 0.3749 - val_loss: 2.1200 - val_acc: 0.5374
Epoch 41/120
312/312 [==============================] - 52s 166ms/step - loss: 3.0592 - acc: 0.4256 - val_loss: 1.8020 - val_acc: 0.6111
Epoch 42/120
312/312 [==============================] - 53s 170ms/step - loss: 2.9205 - acc: 0.4499 - val_loss: 1.8400 - val_acc: 0.6112
Epoch 

Epoch 72/120
312/312 [==============================] - 52s 168ms/step - loss: 2.5262 - acc: 0.5273 - val_loss: 1.4951 - val_acc: 0.6746
Epoch 73/120
312/312 [==============================] - 52s 167ms/step - loss: 2.5088 - acc: 0.5280 - val_loss: 1.4556 - val_acc: 0.6733
Epoch 74/120
312/312 [==============================] - 52s 166ms/step - loss: 2.5666 - acc: 0.5314 - val_loss: 1.4887 - val_acc: 0.6735
Epoch 75/120
312/312 [==============================] - 52s 168ms/step - loss: 2.5416 - acc: 0.5197 - val_loss: 1.4495 - val_acc: 0.6764
Epoch 76/120
312/312 [==============================] - 53s 169ms/step - loss: 2.5008 - acc: 0.5399 - val_loss: 1.4770 - val_acc: 0.6801
Epoch 77/120
312/312 [==============================] - 52s 167ms/step - loss: 2.5509 - acc: 0.5127 - val_loss: 1.4517 - val_acc: 0.6798
Epoch 78/120
312/312 [==============================] - 52s 166ms/step - loss: 2.5497 - acc: 0.5143 - val_loss: 1.4952 - val_acc: 0.6776
Epoch 79/120
312/312 [===================

## evaluate

In [17]:
model_cutmix=load_model("model.cutout+mixup.h5")
loss,acc=evaluate(model_cutmix,x_test,y_test)
print("cutout+mixup : "+prnt.format(loss,acc)+'\n')

model_cutmix=load_model("model_cutout+cutmix.h5")
loss,acc=evaluate(model_cutmix,x_test,y_test)
print("cutout+cutmix : "+prnt.format(loss,acc)+'\n')

10000/10000 [==============================] - 7s 714us/sample - loss: 1.3716 - acc: 0.6971
cutout+mixup : test loss 1.3716  test acc:0.6971

10000/10000 [==============================] - 8s 754us/sample - loss: 1.4316 - acc: 0.6896
cutout+cutmix : test loss 1.4316  test acc:0.6896

